# Introduction

In this notebook, we run a series of database operations using standard Pandas, running on CPU. These values will be logged into MLFlow and used as refecence to compare with the GPU accelerated version using CUDF. 

# About the data

The data we'll be working with is a subset of the [USA 514 Stocks Prices NASDAQ NYSE dataset](https://www.kaggle.com/datasets/olegshpagin/usa-stocks-prices-ohlcv) from Kaggle. This was segmented in differently sized samples, with 5M, 10M, 15M and 20M data entries, and should be set up as an asset (Dataset) called USA_Stocks on the AI Studio project. This code tests whether your dataset is properly setup:

In [1]:
import os.path

if os.path.exists("/home/jovyan/datafabric/USA_Stocks/usa_stocks_5m.parquet") and \
   os.path.exists("/home/jovyan/datafabric/USA_Stocks/usa_stocks_10m.parquet") and \
   os.path.exists("/home/jovyan/datafabric/USA_Stocks/usa_stocks_15m.parquet") and \
   os.path.exists("/home/jovyan/datafabric/USA_Stocks/usa_stocks_20m.parquet"):
    print("Dataset is properly configured")
else:
    print("Dataset is not properly configured. Please, create and download the assets on your project on AI Studio")


Dataset is properly configured


# Analysis in standard Pandas

In the next cells, we will define functions to run different operations in datasets:
  * A function to describe the dataset
  * A function to aggregate results grouped by "ticker" (the identifier of each stock)
  * A function to aggregate by ticker, year and week
  * A function to retrieve a rolling window with a given number of days for each ticker

For each of these functions, the result will be displayed, and the necessary time to run will be logged into MLFlow. These functions will then be applied to the given set of samples in sample_sizes (e.g. [5, 10, 15, 20]). Bigger samples (15M and 20M) might be too heavy depending on the setup of your computer, so we recommend to configure the desired sample sizes according to the available resources:
    

In [2]:
import pandas as pd
import mlflow
import timeit

sample_sizes = [5, 10]
rolling_windows = [7]

In [3]:
import time
def describe(df):
    start_time = time.time()
    result = df.describe
    return time.time() - start_time, result

def simple_aggregation(df):
    start_time = time.time()
    result = df.groupby("ticker").agg({"datetime": ["min", "max", "count"]})
    return time.time() - start_time, result    

def composite_aggregation(df):
    start_time = time.time()
    df[["year", "week", "day"]] = df.datetime.dt.isocalendar()
    result = df.groupby(["ticker", "year", "week"]).agg({"close": ["min", "max"]})
    return time.time() - start_time, result    
    
def rolling_window(df, days):
    start_time = time.time()
    result = df.set_index("datetime").sort_index().groupby("ticker").rolling(f"{days}D").mean().reset_index() 
    return time.time() - start_time, result    
    
    
    

In [5]:
experiment_name = "USA Stock analysis"

mlflow.set_experiment(experiment_name=experiment_name)
for size in sample_sizes:
    with mlflow.start_run(run_name = f"Standard - {size}M") as run:
        mlflow.log_param("Computing", "cpu")
        mlflow.log_param("Dataset size in MB", size)
        df = pd.read_parquet(f"/home/jovyan/datafabric/USA_Stocks/usa_stocks_{size}m.parquet")
        mlflow.log_metric(f"Description time in s", describe(df)[0])
        mlflow.log_metric(f"Simple Aggregation time in s", simple_aggregation(df)[0])
        mlflow.log_metric(f"Composite Aggregation time in s", composite_aggregation(df)[0])
        for window_size in rolling_windows:
           mlflow.log_metric(f"Rolling window {window_size}D time in s", rolling_window(df, window_size)[0])



2024/12/05 20:38:03 INFO mlflow.tracking.fluent: Experiment with name 'USA Stock analysis' does not exist. Creating a new experiment.
